<a href="https://colab.research.google.com/github/ferdouszislam/java-docstring-generator/blob/main/javadoc_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers installation
! pip install transformers datasets evaluate accelerate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=cf7318bf3bca74edaab6f4b117f5ea9e323e7db9d99bdb6273fec23c7073e201
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [2]:
from datasets import load_dataset
from transformers import (AutoTokenizer, DataCollatorForSeq2Seq,
                          AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments,
                          Seq2SeqTrainer, pipeline)
import evaluate
import numpy as np

In [3]:
javadoc_ds = load_dataset(
    'Shuu12121/java-treesitter-dedupe_doc-filtered-dataset',
    split='train[:100]')

sample = javadoc_ds[0]
print(f"features: {sample.keys()}")

# print(f"code:\n {sample['code']}")
# print(f"docstring:\n {sample['docstring']}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004.parquet:   0%|          | 0.00/69.8M [00:00<?, ?B/s]

data/train-00001-of-00004.parquet:   0%|          | 0.00/77.1M [00:00<?, ?B/s]

data/train-00002-of-00004.parquet:   0%|          | 0.00/76.7M [00:00<?, ?B/s]

...

data/test-00000-of-00001.parquet:   0%|          | 0.00/4.49M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1553016 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15165 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17927 [00:00<?, ? examples/s]

features: dict_keys(['code', 'docstring', 'func_name', 'language', 'repo', 'path', 'url', 'license'])


In [4]:
javadoc_ds = javadoc_ds.train_test_split(test_size=0.2)
print(javadoc_ds)

DatasetDict({
    train: Dataset({
        features: ['code', 'docstring', 'func_name', 'language', 'repo', 'path', 'url', 'license'],
        num_rows: 80
    })
    test: Dataset({
        features: ['code', 'docstring', 'func_name', 'language', 'repo', 'path', 'url', 'license'],
        num_rows: 20
    })
})


In [5]:
model_name = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Tokenize dataset

In [6]:
INPUT_PREFIX = "Generate JavaDoc for the function: "

def preprocess_ds(ds):
  inputs = [INPUT_PREFIX + doc for doc in ds["code"]]
  model_inputs = tokenizer(inputs, max_length=512, truncation=True)
  labels = tokenizer(text_target=ds["docstring"], max_length=256,
                      truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [7]:
tokenized_javadoc_ds = javadoc_ds.map(preprocess_ds, batched=True)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

Setup evaluation process

In [9]:
# Load BLEU metric
bleu = evaluate.load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU expects list of references for each prediction
    references = [[label] for label in decoded_labels]

    # Compute BLEU score
    result = bleu.compute(predictions=decoded_preds, references=references)

    # Extract the scalar BLEU score
    bleu_score = result["bleu"]

    # Add generation length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    gen_len = np.mean(prediction_lens)

    return {"eval_bleu": round(bleu_score, 4), "eval_gen_len": round(gen_len, 4)}

Setup model

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Train model

In [ ]:
MODEL_STORE_PATH = "./demis_java_docstr_generator"

training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_STORE_PATH,
    eval_strategy="epoch",
    learning_rate=5e-5,                   # Increase (faster convergence)
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=1,                   # Keep fewer checkpoints
    num_train_epochs=4,                   # Reduce epochs
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
    report_to="none" # Disable wandb reporting
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_javadoc_ds["train"],
    eval_dataset=tokenized_javadoc_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Print evaluation scores

In [ ]:
results = trainer.evaluate()

def print_results(results, model_name):
    print(f"\n{model_name} Results:")
    print("-" * 30)
    print(f"BLEU Score: {results['eval_bleu']:.4f}")
    print(f"Gen Length: {results['eval_gen_len']:.1f}")

print_results(results, "T5-Small Java Docstring")

Save model

In [ ]:
trainer.save_model(MODEL_STORE_PATH)

Inference

In [ ]:
code_sample = """
public boolean isPalindrome(String str) {
    // Remove spaces and convert to lowercase for case-insensitive comparison
    String cleanedStr = str.replaceAll("\\s+", "").toLowerCase();

    int left = 0;
    int right = cleanedStr.length() - 1;

    while (left < right) {
        if (cleanedStr.charAt(left) != cleanedStr.charAt(right)) {
            return false;
        }
        left++;
        right--;
    }

    return true;
}
"""
code_sample = INPUT_PREFIX + code_sample

Inference via pipeline()

In [ ]:
demis_java_docstr_generator = pipeline("summarization",
                                       model=MODEL_STORE_PATH,
                                       tokenizer=MODEL_STORE_PATH)
demis_java_docstr_generator(code_sample)

Manual Inference

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_STORE_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_STORE_PATH)
inputs = tokenizer(code_sample, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
tokenizer.decode(outputs[0], skip_special_tokens=True)